# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [1]:
# importing needed libraries for the entire notebook in one go
# %matplotlib inline
# %matplotlib widget
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from itertools import combinations
import eof
import geemap
import csv
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, box
import glob
import subprocess

In [ ]:
def project_dir(proj_name, operatingsys):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    proj_name = str
        string that contains the name of your project

    operatingsys = str
        'linux' or 'wsl', the linux operating system you are using will impact where data will be save
    """

    if operatingsys == 'linux':
        print('Creating Project Working Directory in your "~" directory')
        work_dir = os.path.join(os.path.expanduser('~'),f'{proj_name}_InSAR')
    elif operatingsys == 'wsl':
        print('Creating Project Working Directory on your Windows Desktop')
        work_dir = os.path.join('/mnt/c/Users/clayc/Desktop',f'{proj_name}_InSAR')

    #creates file on your desktop containing the work of this notebook
    os.makedirs(work_dir, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'interferometry')
    os.makedirs(if_dir, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'time_series')
    os.makedirs(ts_dir, exist_ok=True)
    

    xmls_dir = os.path.join(if_dir,'xmls')
    os.makedirs(xmls_dir, exist_ok=True)
    
    xmldirectories=[]
    for dir in ['topsApp', 'reference', 'secondary']:
        dirpath = os.path.join(xmls_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        xmldirectories.append(dirpath)

    ifdirectories=[]
    for dir in ['reference', 'secondary', 'orbits', 'work']:
        dirpath = os.path.join(if_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        ifdirectories.append(dirpath)

    tsdirectories=[]
    for dir in ['baselines', 'reference', 'merged', 'secondaries', 'mintpy']:
        dirpath = os.path.join(ts_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        if dir == 'merged':
                geomref_dir = os.path.join(dirpath,'geom_reference')
                os.makedirs(geomref_dir, exist_ok=True)

                interfer_dir = os.path.join(dirpath,'interferograms')
                os.makedirs(interfer_dir, exist_ok=True)

                tsdirectories.append(geomref_dir)
                tsdirectories.append(interfer_dir)

        tsdirectories.append(dirpath)

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [96]:
def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i], slc_list[i+2]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet

In [4]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
# function to write the 
def ref_sec_xml(slc_zips_list, slc_zips_dirs):
    for i in range(len(slc_zips_dirs)):
        for j, type in enumerate(['reference', 'secondary']):
            imset = ET.Element('component', name=type)
            safe = ET.SubElement(imset, 'property', name='safe').text = slc_zips_dirs[i]
            out_dir =ET.SubElement(imset, 'property', name='output directory').text = ifdirectories[j]
            orbit_dir =ET.SubElement(imset, 'property', name='orbit directory').text = ifdirectories[2]
            roi = ET.SubElement(imset, 'property', name='region of interest').text = str(isce_aoi)
            tree = ET.ElementTree(imset)
            tree.write(os.path.join(xmldirectories[j+1], f'{slc_zips_list[i][17:25]}{type[:3]}.xml'))

In [ ]:
# function to create the topsApp.xml files for topsInSAR
def topsApp_xml(xmlname, do_iono):
    """
    do_iono = str
        'True' or 'False', does ionospheric correction of the interfoerograms
    """

    data = ET.Element('topsApp')
    tinsar = ET.SubElement(data, 'component', name='topsinsar')
    snsr_name = ET.SubElement(tinsar, 'property', name='Sensor name').text = 'SENTINEL1'
    ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
    xml_file = ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{xmlname[:8]}ref.xml")
    sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
    xml_file = ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{xmlname[9:17]}sec.xml")
    
    swathnums = ET.SubElement(tinsar, 'property', name='swaths').text=str([3])
    azi_looks = ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(1)   # for sentinel-1, azimuth resolution is 22m        
    range_looks = ET.SubElement(tinsar, 'property', name='range looks').text = str(5)   # for sentinel-1, range resolution is 2.7 to 3.5m, wan this to be larger than azimuth looks                  
    phasefilt = ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)                        
    
    unwrap_yn = ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True) # or False         
    unwrap_name = ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf' # grass, icu, snaphu, snaphu_mcf, downsample_snap         

    if do_iono == 'True':
        do_iono_corr = ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = do_iono
        apply_iono_corr = ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = do_iono

        #choose from: ['subband', 'rawion', 'grd2ion', 'filt_gaussian', 'ionosphere_shift', 'ion2grd', 'esd']
        start_iono_corr = ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'filt_gaussian'
        end_iono_corr = ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'
        burst_params = ET.SubElement(tinsar, 'property')

        iono_height = ET.SubElement(tinsar, 'property', name='height of ionosphere layer in km').text = str(200.0)
        iono_filt = ET.SubElement(tinsar, 'property', name="apply polynomial fit before filtering ionosphere phase").text = str(True)
        max_iono_filt_wind_phase = ET.SubElement(tinsar, 'property', name="maximum window size for filtering ionosphere phase").text = str(200)
        min_iono_filt_wind_phase = ET.SubElement(tinsar, 'property', name="minimum window size for filtering ionosphere phase").text = str(100)
        max_iono_filt_wind_azishift = ET.SubElement(tinsar, 'property', name="maximum window size for filtering ionosphere azimuth shift").text = str(150)
        min_iono_filt_wind_azishift = ET.SubElement(tinsar, 'property', name="minimum window size for filtering ionosphere azimuth shift").text = str(75)

        #0: no correction
        #1: use mean value of a burst
        #2: use full burst
        correct_iono_azishift_err = ET.SubElement(tinsar, 'property', name='correct phase error caused by ionosphere azimuth shift').text = str(2)

    geocode = ET.SubElement(tinsar, 'property', name='geocode bounding box').text = str(isce_aoi)
            
    tree = ET.ElementTree(data)
    tree.write(os.path.join(xmldirectories[0], xmlname))

In [19]:
# took these from UNAVCO for plotting the results from topsApp

# Utility to plot a 2D array
def plotdata(GDALfilename, band=1,
             title=None,colormap='gray',
             aspect=1, background=None,
             datamin=None, datamax=None,
             interpolation='nearest',
             nodata = None,
             draw_colorbar=True, colorbar_orientation="horizontal"):
    
    # Read the data into an array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    try:
        if nodata is not None:
            data[data == nodata] = np.nan
    except:
        pass
        
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    
    # clearing the data
    data = None

# Utility to plot interferograms
def plotcomplexdata(GDALfilename,
                    title=None, aspect=1,
                    datamin=None, datamax=None,
                    interpolation='nearest',
                    draw_colorbar=None, colorbar_orientation="horizontal"):
    # Load the data into numpy array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    try:
        slc[slc==0]=np.nan
    except:
        pass

    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc), vmin = datamin, vmax=datamax,
                   cmap='gray', extent=[xmin,xmax,ymin,ymax],
                   interpolation=interpolation)
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc), cmap='rainbow',
                    vmin=-np.pi, vmax=np.pi,
                    extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2, orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    
    # clearing the data
    slc = None

# Utility to plot multiple similar arrays
def plotstackdata(GDALfilename_wildcard, band=1,
                  title=None, colormap='gray',
                  aspect=1, datamin=None, datamax=None,
                  interpolation='nearest',
                  draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(band).ReadAsArray()   
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass            
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show() 

    # clearing the data
    data = None

# Utility to plot multiple simple complex arrays
def plotstackcomplexdata(GDALfilename_wildcard,
                         title=None, aspect=1,
                         datamin=None, datamax=None,
                         interpolation='nearest',
                         draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    print(GDALfilenames)
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(1).ReadAsArray()
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass              
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(data), vmin=datamin, vmax=datamax,
                   cmap='gray', interpolation='nearest')
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(data), cmap='rainbow',
                            interpolation='nearest')
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show() 
    
    # clearing the data
    data = None

# Establish working directroy and data paths

In [206]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'SabineRS'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name, 'linux')

Creating Project Working Directory in your "~" directory


In [207]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

# Get bbox for your area of interest in the SLC images, faster processing with less data

In [48]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
# m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [49]:
# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(m.draw_features[0].getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
isce_aoi_polygon = Polygon(vertices)

#create isce compatibile bbox
isce_aoi = [min([easting for northing, easting in vertices[:]]), max([easting for northing, easting in vertices[:]]), min([northing for northing, easting in vertices[:]]), max([northing for northing, easting in vertices[:]])]
print(isce_aoi)

[29.825157, 30.308874, -93.547211, -93.126984]


In [ ]:
# isce_aoi = [29.88709, 30.151659, -93.538971, -93.313751]

# Download water mask image for aoi

In [ ]:
os.chdir(work_dir)

snaphuexport_cmd = [
    '/home/wcc/tools/miniforge/envs/isce/lib/python3.8/site-packages/isce/applications/wbd.py',
    isce_aoi[0],
    isce_aoi[1],
    isce_aoi[2],
    isce_aoi[3]
]

wbd_file = os.path.expandvars(os.path.join(work_dir,'*.wbd'))

# Run the command
try:
    result = subprocess.run(snaphuexport_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully!")
except subprocess.CalledProcessError as e:
    print(f"Command failed with error code {e.returncode}")
    print(f"Error message: {e.stderr}")

# Get s-1 orbit files (.EOF) using sentineleof library

In [50]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create Sentinel-1 SLC .xml files
- One reference and one secondary for each image

In [210]:
ref_sec_xml(slc_zips_list, slc_zips_dirs)

# Create topsApp.xml file
- look into computing baselines, running dense offsets for wrapped and unwrapped ifgs, water masking, shadow masking, and ionosphere correction
- one for each interferogram in each triplet

in order of importance
1. automation for all SLCs (including removing old data and organizing to mintpy standards)
2. computing baselines
3. adding in dense offsets for both ifgs
4. shadow masking
5. water masking
6. iono correction (may increase in importance if I start to work with L or S-band. NISAR)

In [ ]:
# test with below for more options
def topsApp_xml(xmlname, do_iono, water_mask):
    """
    Create a topsApp XML configuration file for Sentinel-1 InSAR processing.
    
    Parameters:
        xmlname (str): Name of the output XML file.
        do_iono (str): 'True' or 'False', to enable or disable ionospheric correction.
        water_mask (str): Path to the water mask file from get_water_mask.ipynb
    """
    data = ET.Element('topsApp')
    tinsar = ET.SubElement(data, 'component', name='topsinsar')

    # Sensor and data settings
    ET.SubElement(tinsar, 'property', name='Sensor name').text = 'SENTINEL1'
    ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
    ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{xmlname[:8]}ref.xml")
    sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
    ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{xmlname[9:17]}sec.xml")

    # General processing settings
    ET.SubElement(tinsar, 'property', name='swaths').text = str([3])
    ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(1)
    ET.SubElement(tinsar, 'property', name='range looks').text = str(5)
    ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)
    ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True)
    ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf'
    ET.SubElement(tinsar, 'property', name='geocode bounding box').text = str(isce_aoi)

    # Add water masking
    ET.SubElement(tinsar, 'property', name='apply water mask').text = str(True)
    ET.SubElement(tinsar, 'property', name='water mask file name').text = water_mask # from get_water_mask.ipynb

    # Add dense offsets
    ET.SubElement(tinsar, 'property', name='do dense offsets').text = str(True)
    ET.SubElement(tinsar, 'property', name='dense offset window width').text = str(64)
    ET.SubElement(tinsar, 'property', name='dense offset window height').text = str(64)
    ET.SubElement(tinsar, 'property', name='dense offset search width').text = str(20)
    ET.SubElement(tinsar, 'property', name='dense offset search height').text = str(20)
    ET.SubElement(tinsar, 'property', name='dense offset snr threshold').text = str(8.0)

    # Add ionospheric correction
    if do_iono == 'True':
        ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = do_iono
        ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = do_iono
        #choose from: ['subband', 'rawion', 'grd2ion', 'filt_gaussian', 'ionosphere_shift', 'ion2grd', 'esd']
        ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'subband'
        ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'
        ET.SubElement(tinsar, 'property', name='height of ionosphere layer in km').text = str(200.0)
        ET.SubElement(tinsar, 'property', name='apply polynomial fit before filtering ionosphere phase').text = str(True)
        ET.SubElement(tinsar, 'property', name='maximum window size for filtering ionosphere phase').text = str(200)
        ET.SubElement(tinsar, 'property', name='minimum window size for filtering ionosphere phase').text = str(100)
        
        #0: no correction
        #1: use mean value of a burst
        #2: use full burst
        ET.SubElement(tinsar, 'property', name='correct phase error caused by ionosphere azimuth shift').text = str(2)
    
    # Save XML
    tree = ET.ElementTree(data)
    tree.write(os.path.join(xmldirectories[0], xmlname))

In [226]:
# stores these in xmldirectories[0]

do_iono = 'False'

for i, m in enumerate(slc_zips_list):
    if i < len(slc_zips_list) - 4:
        for date in [f'{m[17:25]}_{slc_zips_list[i+1][17:25]}', f'{m[17:25]}_{slc_zips_list[i+2][17:25]}', f'{m[17:25]}_{slc_zips_list[i+3][17:25]}']:
            topsApp_xml(f'{date}topsApp.xml', do_iono)

    elif i == len(slc_zips_list) - 3:
        for date in [f'{m[17:25]}_{slc_zips_list[i+1][17:25]}', f'{m[17:25]}_{slc_zips_list[i+2][17:25]}']:
            topsApp_xml(f'{date}topsApp.xml', do_iono)


    elif i == len(slc_zips_list) - 2:
        topsApp_xml(f"{m[17:25]}_{slc_zips_list[i+1][17:25]}topsApp.xml", do_iono)

    elif i == len(slc_zips_list) - 1:
        break

# topsApp interferometry (finally!)

In [227]:
# should import isce toolbox and print the version of it here
# if nothing prints or nothing imports, you need to reinstall isce2

import isce
isce.version.release_version

'2.6.3'

In [228]:
# this is used to change directory into the storage directory for all interferograms
# should be used, else will save results to github repo
os.chdir(ifdirectories[3])
print(ifdirectories[3])

from topsApp import TopsInSAR

/home/wcc/SabineRS_InSAR/interferometry/work


In [ ]:
# check 

for xml in sorted(os.listdir(xmldirectories[0]), key=lambda x: datetime.strptime(x[:8], '%Y%m%d'))[:3]:
    xml_path = os.path.join(xmldirectories[0] , xml)
    insar = TopsInSAR(name="topsApp", cmdline=xml_path)
    insar.configure()
    insar.run()

2024-12-09 13:58:39,988 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
None
The currently supported sensors are:  ['SENTINEL1']
Input XML files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.SAFE/annotation/s1a-iw3-slc-vv-20191001t001839-20191001t001904-029258-035306-006.xml']
Input TIFF files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.SAFE/measurement/s1a-iw3-slc-vv-20191001t001839-20191001t001904-029258-035306-006.tiff']
Manifest files:  ['/vsizip//home/wcc/Desktop/SabineRS/Sentinel-1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   498    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   283    0   283    0     0    643      0 --:--:-- --:--:-- --:--:--  3076
  0   267    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 6939k  100 6939k    0     0  3326k      0  0:00:02  0:00:02 --:--:-- 9074k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N31W093.SRTMGL1.hgt.zip


100 6966k  100 6966k    0     0  4111k      0  0:00:01  0:00:01 --:--:-- 4109k


command = curl -n  -L -c $HOME/.earthdatacookie -b $HOME/.earthdatacookie -k -f -O https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N30W094.SRTMGL1.hgt.zip


...

API open (R): demLat_N29_N32_Lon_W094_W092.dem
API close:  demLat_N29_N32_Lon_W094_W092.dem
Writing geotrans to VRT for demLat_N29_N32_Lon_W094_W092.dem
GDAL open (R): demLat_N29_N32_Lon_W094_W092.dem.vrt
API open (WR): demLat_N29_N32_Lon_W094_W092.dem.wgs84

    << Geoid Correction I2 SRTM>>

Jet Propulsion Laboratory - Radar Science and Engineering


Sampling Geoid at grid points -  Longitude Samples:    23 Latitude Lines:    33
Corner Geoid Heights (m) =  -25.27 -26.70 -27.10 -26.85

Correcting data to geoid height...

At line:      512
At line:     1024
At line:     1536
At line:     2048
At line:     2560
At line:     3072
At line:     3584
At line:     4096
At line:     4608
At line:     5120
At line:     5632
At line:     6144
At line:     6656
At line:     7168
At line:     7680
At line:     8192
At line:     8704
At line:     9216
At line:     9728
At line:    10240
At line:    10752
GDAL close: demLat_N29_N32_Lon_W094_W092.dem.vrt
API close:  demLat_N29_N32_Lon_W094_W092.dem.

line: 12224


number of lines written to file: 12240
Rescaling magnitude


line: 12239

GDAL close: merged/topophase.flat.vrt
API close:  merged/filt_topophase.flat
GDAL open (R): merged/filt_topophase.flat.vrt
API open (WR): merged/phsig.cor


In [ ]:
# can run these steps separately if you want more info on where process may be failing

# insar = TopsInSAR(name="topsApp", cmdline=xml_path)
# insar.configure()
# insar.startup()
# insar.runComputeBaseline()
# insar.verifyDEM()
# insar.verifyGeocodeDEM()
# insar.runTopo()
# insar.runSubsetOverlaps()
# insar.runCoarseOffsets()
# insar.runCoarseResamp()
# insar.runOverlapIfg()
# insar.runPrepESD()
# insar.runESD()
# insar.runRangeCoreg()
# insar.runFineOffsets()
# insar.runFineResamp()
# insar.runIon()
# insar.runBurstIfg()
# insar.runMergeBursts()
# #add dense offsets here for wrapped phase if you'd like
# insar.runFilter()
# insar.runUnwrapper()    # can also run 2stage unwrapper and dump to pickle if you want
# insar.runGeocode(insar.geocode_list, insar.do_unwrap, insar.geocode_bbox)
# insar.runDenseOffsets()
# insar.runOffsetFilter()
# insar.runGeocode(insar.off_geocode_list, True, insar.geocode_bbox, True)
# insar.endup()

2024-12-09 13:51:04,043 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 


Steps include:
- runPreprocessor
- runComputeBaseline
- verifyDEM
- verifyGeocodeDEM
- runTopo
- runSubsetOverlaps
- runCoarseOffsets
- runCoarseResamp
- runOverlapIfg
- runPrepESD
- runESD
- runRangeCoreg
- runFineOffsets
- runFineResamp
- runIon
- runBurstIfg
- runMergeBursts
- runFilter
- runGeocode
- runDenseOffsets
- runOffsetFilter

# Generate baseline file for interferograms
- uses isce2/contrib/stack/topsStack/computeBaselin.py, you will need to provide your path

In [40]:
ifg_date = f'{test_triplet[0][0][17:25]}_{test_triplet[0][1][17:25]}'
ifg_date

'20191001_20191013'

In [ ]:
def generate_baselines(date, ref_dir, sec_dir, computeBaselinepath):
    

In [42]:
tsdirectories[0]

'/home/wcc/SabineRS_InSAR/time_series/baseline'

# Arrange file directories for use in MintPy
- https://github.com/insarlab/MintPy-tutorial/blob/main/workflows/smallbaselineApp.ipynb
- reference the Fernandina example set
- for 'reference' and 'merged/reference' files, only needed from first ifg

# NEED TO FIGURE THIS OUT FIRST, AUTOMATION ALREADY WORKS BUT OVERWRITES ITSELF

In [ ]:
# mintpy needs

# mintpy.load.processor        = isce
# ##---------for ISCE only:
# mintpy.load.metaFile         = $DATA_DIR/GalapagosSenDT128/reference/IW*.xml
# mintpy.load.baselineDir      = $DATA_DIR/GalapagosSenDT128/baselines
# ##---------interferogram datasets:
# mintpy.load.unwFile          = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.unw
# mintpy.load.corFile          = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.cor
# mintpy.load.connCompFile     = $DATA_DIR/GalapagosSenDT128/merged/interferograms/*/filt_*.unw.conncomp
# ##---------geometry datasets:
# mintpy.load.demFile          = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/hgt.rdr
# mintpy.load.lookupYFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/lat.rdr
# mintpy.load.lookupXFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/lon.rdr
# mintpy.load.incAngleFile     = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/los.rdr
# mintpy.load.azAngleFile      = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/los.rdr
# mintpy.load.shadowMaskFile   = $DATA_DIR/GalapagosSenDT128/merged/geom_reference/shadowMask.rdr



once we have all the needed files, I will delete the unneeded files produced during processing to save storage

In [ ]:
# maybe put something here that will move the files generated
# put them into the example directory that Mintpy needs for topsStack



# Working to make it so that the above saves into needed file directories for Mintpy

https://mintpy.readthedocs.io/en/stable/dir_structure/

Essentially just needs another function for:
1. runnings prep_isce.py on the data produced above
2. move all needed data created from ISCE2 into the tsdirectories according to the above link

(this may need to be done in the mintpy environment, so I may end up including this in the time series ipynb, or a short .py to run on it's own)